In [ ]:
#数据加载和处理
#假设你有一个CSV文件（stocks.csv），其中包含多支股票的历史数据，包括日期、开盘价、收盘价、最高价、最低价、成交量等。这里我们选择以收盘价进行分析。

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# 加载数据
data = pd.read_csv('stocks.csv')
data['Date'] = pd.to_datetime(data['Date'])

# 选择股票的收盘价作为目标变量
stocks = data[['Date', 'Stock1_Close', 'Stock2_Close', 'Stock3_Close']]
stocks.set_index('Date', inplace=True)

# 简单数据清理：检查缺失值
stocks.isnull().sum()
stocks = stocks.dropna()


In [ ]:
#特征工程
#我们将使用过去7天的收盘价作为特征，预测未来1天的收盘价。

In [ ]:
# 生成过去7天的特征
for stock in ['Stock1_Close', 'Stock2_Close', 'Stock3_Close']:
    for i in range(1, 8):
        stocks[f'{stock}_lag_{i}'] = stocks[stock].shift(i)

stocks = stocks.dropna()  # 删除缺失值


In [ ]:
#划分训练集和测试集

#将数据分为训练集和测试集，80%用于训练，20%用于测试。

In [ ]:
# 选择特征和目标
features = [col for col in stocks.columns if 'lag' in col]
target = 'Stock1_Close'  # 假设我们预测Stock1的收盘价

X = stocks[features]
y = stocks[target]

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)


In [ ]:
#模型训练与评估

#使用线性回归进行预测，评估模型的效果

In [ ]:
# 初始化并训练模型
model = LinearRegression()
model.fit(X_train, y_train)

# 预测并评估
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

# 可视化结果
plt.plot(y_test.index, y_test.values, label='True')
plt.plot(y_test.index, y_pred, label='Predicted')
plt.legend()
plt.title('Stock1 Close Price Prediction')
plt.show()


In [ ]:
#投资组合优化

#我们使用现代投资组合理论（均值-方差优化）来构建一个最优投资组合，目标是最大化预期收益并最小化风险。

In [ ]:
from scipy.optimize import minimize

# 假设我们有3只股票
returns = stocks.pct_change().dropna()  # 计算每日收益率
mean_returns = returns.mean()
cov_matrix = returns.cov()

# 投资组合优化：最小化风险（方差）
def portfolio_variance(weights, mean_returns, cov_matrix):
    return np.dot(weights.T, np.dot(cov_matrix, weights))

# 投资组合的约束：权重和为1，且每只股票的权重非负
constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1})
bounds = tuple((0, 1) for _ in range(len(mean_returns)))

# 初始猜测：等权重
initial_guess = [1.0 / len(mean_returns)] * len(mean_returns)

# 优化
result = minimize(portfolio_variance, initial_guess, args=(mean_returns, cov_matrix), method='SLSQP', bounds=bounds, constraints=constraints)

# 输出最优组合
optimal_weights = result.x
print(f'Optimal Portfolio Weights: {optimal_weights}')

# 计算最优组合的预期收益和风险
optimal_return = np.dot(optimal_weights, mean_returns)
optimal_risk = np.sqrt(portfolio_variance(optimal_weights, mean_returns, cov_matrix))
print(f'Optimal Portfolio Return: {optimal_return}')
print(f'Optimal Portfolio Risk (Standard Deviation): {optimal_risk}')


In [ ]:
#总结与展示

#将最终的投资组合权重和预测结果输出，并在GitHub上提交。

In [ ]:
# 保存结果到GitHub代码库
stocks['Predicted_Stock1_Close'] = model.predict(X)
stocks.to_csv('stocks_with_predictions.csv')

# 展示优化后的投资组合权重
print(f"Optimal Weights: {optimal_weights}")
print(f"Portfolio Expected Return: {optimal_return}")
print(f"Portfolio Risk: {optimal_risk}")

# 创建一个展示图表（可以展示股票预测和投资组合效果）
plt.figure(figsize=(10, 6))
plt.plot(stocks.index, stocks['Stock1_Close'], label='Stock1 Actual')
plt.plot(stocks.index, stocks['Predicted_Stock1_Close'], label='Stock1 Predicted')
plt.legend()
plt.show()


In [ ]:
"""这个示例展示了如何使用Python完成以下任务：

数据加载与处理：加载股票数据，处理缺失值，并生成特征。
回归分析：使用线性回归预测未来股票价格。
投资组合优化：使用现代投资组合理论（均值-方差优化）找到最优投资组合。"""